In [1]:
import json
import pandas as pd
import spacy
import re
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import codecs
import pathlib
from pathlib import Path
import os
import gensim
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.corpora import Dictionary
import pyLDAvis.gensim_models
import pyLDAvis.sklearn
from sklearn.decomposition import LatentDirichletAllocation
from functions import *
import os

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/laurapaulsen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:

nlp = spacy.load("da_core_news_sm")
stops_path = '../../stop_words.txt'
#file_path = '../json/contribute_FB_01122112.json'
file_path = '../json/contribute_FB_10101011.json'
# file_path = '../json/FB_10101111010.json'
ID = file_path.split('_')[-1].split('.')[0]

file = open(stops_path,"r+")
stops = file.read().split()

data = load_json(file_path)

extracted_posts = extract_posts(data)

save_txt(extracted_posts, '../extracted_fb_posts', ID + '.txt')

decoded_data = decode('../extracted_fb_posts/' + ID +'.txt')

no_mentions = remove_mentions(decoded_data)

no_empty_str = [string for string in  no_mentions if string != ""]

no_empty_str = change_letter(no_empty_str)

cleaned_data = clean_text(no_empty_str)

cleaned_data = [string for string in cleaned_data if string != ""]

tokenized_data = tokenize(cleaned_data)

lemmatized_data = lemmatize(tokenized_data, nlp = nlp)

# After lemmatization we again have words with special letters 
lemmatized_data = change_char(lemmatized_data)

no_stops = remove_stops(lemmatized_data, stops)

no_stops = [l for l in no_stops if len(l) != 0]

no_stops = rem_single_char(no_stops)

bigram = gensim.models.Phrases(no_stops)

dictionary = Dictionary(no_stops)
corpus = [dictionary.doc2bow(text) for text in no_stops]

max_topics = set_max_topics(no_stops)
min_topics = 2
step = 2
x = range(min_topics, max_topics, step)

coherence_values = compute_c_v(dictionary = dictionary, 
                               corpus = corpus, 
                               texts = no_stops, 
                               min_topics = min_topics, 
                               max_topics = max_topics, 
                               step = step)


best_result_index = coherence_values.index(max(coherence_values))


ldamodel = LdaModel(corpus = corpus, 
                    num_topics = x[best_result_index], 
                    id2word = dictionary,
                    update_every = 1,
                    passes = 10,
                    per_word_topics = True)


model = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)

save_model(model, '../html', ID + '.html')

# saving preprocessed data to .txt for generic model
if 'contribute' in file_path:
    n = 20
    if len(no_stops) > n:
        data_generic = no_stops[0:n]
    else:
        data_generic = no_stops

    if not os.path.isdir('../generic_model'):
        os.mkdir('../generic_model')
    
    if not os.path.isfile('../generic_model/facebook_generic.txt'):
        with open('../generic_model/facebook_generic.txt', 'w') as fp:
            pass

    # Open a file with access mode 'a'
    file_object = open('../generic_model/facebook_generic.txt', 'a')
    # Append no_stops at the end of file
    file_object.write(str(data_generic)[1:-1])
    # Close the file
    file_object.close()

/usr/local/Caskroom/miniconda/base/envs/methods3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/usr/local/Caskroom/miniconda/base/envs/methods3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/Caskroom/miniconda/base/envs/methods3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/Caskroom/miniconda/base/envs/methods3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation 

In [3]:
# removing files
if os.path.exists(file_path):
  os.remove(file_path)

if os.path.exists( '../extracted_fb_posts' + ID + '.txt'):
  os.remove( '../extracted_fb_posts' + ID + '.txt')